In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alici\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [5]:
from imblearn.over_sampling import SMOTE
from collections import Counter

In [6]:
from sklearn.model_selection import GridSearchCV

In [7]:
def createdfs(i):
    df_train = pd.read_csv(r"C:\Users\alici\Documents\tcc\v3\particoes\b5\k_"+str(i)+"/train.csv")
    df_test = pd.read_csv(r"C:\Users\alici\Documents\tcc\v3\particoes\b5\k_"+str(i)+"/test.csv")
    df_train["Transf_Text"] = df_train["Text"].apply((lambda x: (str(x)).split("#")))
    df_test["Transf_Text"] = df_test["Text"].apply((lambda x: (str(x)).split("#")))
    return df_train, df_test

In [8]:
def remove_stopwords(texts):
    result = []
    for text in texts:
        all_words = text.split(" ")
        clean_text = [i for i in all_words if i not in stopwords and i!=""]
        result.append(" ".join(clean_text))
    return result

In [9]:
def getstringtext(df):
    texts = df["CleanText"].to_list()
    string_text = []
    for l in texts:
        res = " ".join(l)
        res = res.replace("\"", "")
        string_text.append(res)
    return string_text

In [10]:
def getmetrics(y_test, y_pred):
    mc = metrics.confusion_matrix(y_test, y_pred)
    a = metrics.accuracy_score(y_test, y_pred)
    p = metrics.precision_score(y_test, y_pred, average=None)
    r = metrics.recall_score(y_test, y_pred, average=None)
    f1 = metrics.f1_score(y_test, y_pred, average=None)
    return mc, a, p, r, f1

In [11]:
def genderGroups(text):
    if(text == "f"):
        return 1
    else:
        return 0

In [12]:
def mean(ages_list):
    soma = [0, 0, 0]
    for a1, a2, a3 in ages_list:
        soma[0] += a1
        soma[1] += a2
        soma[2] += a3
    return [(soma[0]/len(ages_list)), (soma[1]/len(ages_list)), (soma[2]/len(ages_list))]

In [13]:
def formatTime(seg):
    min = seg//60
    seg = seg % 60
    return str(min)+"min"+str(round(seg, 2))+"s"

In [15]:
# GridSearch para genero
import json
import time

best_param_dict = dict()

start = time.time()

grid = dict()
grid["C"] = np.logspace(-3,3,7).tolist()
grid["penalty"] = ["l1", "l2"]
best_param_dict["grid_values"] = grid

for i in range(10):
    df_train_gender, df_test_gender = createdfs(i+1)
    df_train_gender['CleanText'] = df_train_gender['Transf_Text'].apply(remove_stopwords)
    df_test_gender['CleanText'] = df_test_gender['Transf_Text'].apply(remove_stopwords)
    df_train_gender["GenderClass"] = df_train_gender["ActualGender"].apply(genderGroups)
    df_test_gender["GenderClass"] = df_test_gender["ActualGender"].apply(genderGroups)
    train_texts_gender = getstringtext(df_train_gender)
    test_texts_gender = getstringtext(df_test_gender)
    tfvec = TfidfVectorizer(max_features = 3000)
    tfvec.fit(train_texts_gender)
    tdf_train = tfvec.transform(train_texts_gender).toarray().tolist()
    tdf_test = tfvec.transform(test_texts_gender).toarray().tolist()
    gender_train = df_train_gender["GenderClass"].to_list()
    gender_test = df_test_gender["GenderClass"].to_list()

    logreg=LogisticRegression()
    logreg_cv=GridSearchCV(logreg,grid,cv=10, scoring='f1_macro')
    logreg_cv.fit(tdf_train,gender_train)
    
    param_dict = dict()
    param_dict["params"] =  logreg_cv.best_params_
    param_dict["accuracy"] = logreg_cv.best_score_
    best_param_dict["k_"+str(i+1)] = param_dict

    print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
    print("accuracy :",logreg_cv.best_score_)

end = time.time()
best_param_dict["time"] = formatTime(end-start)

with open(r'C:\Users\alici\Documents\tcc\v3\melhores_param\b5\gender.json', 'w') as f:
    json.dump(best_param_dict, f, indent=4)
    

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

tuned hpyerparameters :(best parameters)  {'C': 100.0, 'penalty': 'l2'}
accuracy : 0.8854908623308357


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

tuned hpyerparameters :(best parameters)  {'C': 100.0, 'penalty': 'l2'}
accuracy : 0.8777740877858025


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

...

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

tuned hpyerparameters :(best parameters)  {'C': 1000.0, 'penalty': 'l2'}
accuracy : 0.8688816040538538


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

tuned hpyerparameters :(best parameters)  {'C': 1000.0, 'penalty': 'l2'}
accuracy : 0.8708714111949977


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
70 fits failed out of a total of 140.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
----------------------------------------------------------------

In [16]:
import json

In [17]:
with open(r'C:\Users\alici\Documents\tcc\v3\melhores_param\b5\gender.json', encoding='utf-8') as f:
    gender_param = json.load(f)

In [18]:
# GridSearch para idade (juntos)
import json
import time

best_param_dict = dict()

start = time.time()

grid = dict()
grid["C"] = np.logspace(-3,3,7).tolist()
grid["penalty"] = ["l1", "l2", "none"]
best_param_dict["grid_values"] = grid

for i in range(10):
    df_train_gender, df_test_gender = createdfs(i+1)

    df_train_gender['CleanText'] = df_train_gender['Transf_Text'].apply(remove_stopwords)
    df_test_gender['CleanText'] = df_test_gender['Transf_Text'].apply(remove_stopwords)

    df_train_gender["GenderClass"] = df_train_gender["ActualGender"].apply(genderGroups)
    df_test_gender["GenderClass"] = df_test_gender["ActualGender"].apply(genderGroups)

    train_texts_gender = getstringtext(df_train_gender)
    test_texts_gender = getstringtext(df_test_gender)

    tfvec = TfidfVectorizer(max_features = 3000)
    tfvec.fit(train_texts_gender)
    tdf_train = tfvec.transform(train_texts_gender).toarray().tolist()
    tdf_test = tfvec.transform(test_texts_gender).toarray().tolist()

    age_train = df_train_gender["AgeClass"].to_list()
    age_test = df_test_gender["AgeClass"].to_list()

    logreg=LogisticRegression()
    logreg_cv=GridSearchCV(logreg,grid,cv=10, scoring='f1_macro')
    logreg_cv.fit(tdf_train, age_train)
    
    param_dict = dict()
    param_dict["params"] =  logreg_cv.best_params_
    param_dict["accuracy"] = logreg_cv.best_score_
    best_param_dict["k_"+str(i+1)] = param_dict

    print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
    print("accuracy :",logreg_cv.best_score_)

end = time.time()
best_param_dict["time"] = formatTime(end-start)

with open(r'C:\Users\alici\Documents\tcc\v3\melhores_param\b5\age.json', 'w') as f:
    json.dump(best_param_dict, f, indent=4)

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1

tuned hpyerparameters :(best parameters)  {'C': 1000.0, 'penalty': 'l2'}
accuracy : 0.5752763245678227


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1

tuned hpyerparameters :(best parameters)  {'C': 10.0, 'penalty': 'l2'}
accuracy : 0.5744494862510878


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1

...

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1

tuned hpyerparameters :(best parameters)  {'C': 10.0, 'penalty': 'l2'}
accuracy : 0.5636596091338776


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1

tuned hpyerparameters :(best parameters)  {'C': 1000.0, 'penalty': 'l2'}
accuracy : 0.540978986921835


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
import json
import time

best_param_dict = dict()

start = time.time()

grid = dict()
grid["C"] = np.logspace(-3,3,7).tolist()
grid["penalty"] = ["l1", "l2", "none"]
best_param_dict["grid_values"] = grid

for i in range(10):
    df_train_gender, df_test_gender = createdfs(i+1)

    df_train_gender['CleanText'] = df_train_gender['Transf_Text'].apply(remove_stopwords)
    df_test_gender['CleanText'] = df_test_gender['Transf_Text'].apply(remove_stopwords)

    df_train_gender["GenderClass"] = df_train_gender["ActualGender"].apply(genderGroups)
    df_test_gender["GenderClass"] = df_test_gender["ActualGender"].apply(genderGroups)

    train_texts_gender = getstringtext(df_train_gender)
    test_texts_gender = getstringtext(df_test_gender)

    tfvec = TfidfVectorizer(max_features = 3000)
    tfvec.fit(train_texts_gender)
    tdf_train = tfvec.transform(train_texts_gender).toarray().tolist()
    tdf_test = tfvec.transform(test_texts_gender).toarray().tolist()

    gender_train = df_train_gender["GenderClass"].to_list()
    gender_test = df_test_gender["GenderClass"].to_list()

    logisticRegr_gender = LogisticRegression(penalty=gender_param["k_"+str(i+1)]["params"]["penalty"], C=gender_param["k_"+str(i+1)]["params"]["C"])
    logisticRegr_gender.fit(tdf_train, gender_train)

    pred_gender_test=logisticRegr_gender.predict(tdf_test)

    df_test_gender["PredictGender"] = pred_gender_test

    df_train_fem = df_train_gender[df_train_gender["GenderClass"]==1]
    df_train_masc = df_train_gender[df_train_gender["GenderClass"]==0]

    df_test_fem = df_test_gender[df_test_gender["PredictGender"]==1]
    df_test_masc = df_test_gender[df_test_gender["PredictGender"]==0]

    train_texts_fem = getstringtext(df_train_fem)
    test_texts_fem = getstringtext(df_test_fem)
    tfvec_fem = TfidfVectorizer(max_features = 3000)
    tfvec_fem.fit(train_texts_fem)
    tdf_train_fem = tfvec_fem.transform(train_texts_fem).toarray().tolist()
    tdf_test_fem = tfvec_fem.transform(test_texts_fem).toarray().tolist()
    age_train_fem = df_train_fem["AgeClass"].to_list()
    age_test_fem = df_test_fem["AgeClass"].to_list()

    logreg=LogisticRegression()
    logreg_cv=GridSearchCV(logreg,grid,cv=10, scoring='f1_macro')
    logreg_cv.fit(tdf_train_fem, age_train_fem)
    
    param_dict = dict()
    param_dict["params"] =  logreg_cv.best_params_
    param_dict["accuracy"] = logreg_cv.best_score_
    best_param_dict["k_"+str(i+1)] = param_dict

    print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
    print("accuracy :",logreg_cv.best_score_)

end = time.time()
best_param_dict["time"] = formatTime(end-start)

with open(r'C:\Users\alici\Documents\tcc\v3\melhores_param\b5\age_fem.json', 'w') as f:
    json.dump(best_param_dict, f, indent=4)

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1

tuned hpyerparameters :(best parameters)  {'C': 100.0, 'penalty': 'l2'}
accuracy : 0.5931831706546113


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1

tuned hpyerparameters :(best parameters)  {'C': 100.0, 'penalty': 'l2'}
accuracy : 0.6093842145508812


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Pytho

...

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Pytho

tuned hpyerparameters :(best parameters)  {'C': 0.001, 'penalty': 'none'}
accuracy : 0.5904703279176964


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Pytho

tuned hpyerparameters :(best parameters)  {'C': 0.001, 'penalty': 'none'}
accuracy : 0.5805222735018132


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
70 fits failed out of a total of 210.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
70 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py", line 1094, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "c:\Users\alici\AppData\Local\Programs\Python

In [21]:
import json
import time

best_param_dict = dict()

start = time.time()

grid = dict()
grid["C"] = np.logspace(-3,3,7).tolist()
grid["penalty"] = ["l1", "l2", "none"]
best_param_dict["grid_values"] = grid

for i in range(10):
    df_train_gender, df_test_gender = createdfs(i+1)

    df_train_gender['CleanText'] = df_train_gender['Transf_Text'].apply(remove_stopwords)
    df_test_gender['CleanText'] = df_test_gender['Transf_Text'].apply(remove_stopwords)

    df_train_gender["GenderClass"] = df_train_gender["ActualGender"].apply(genderGroups)
    df_test_gender["GenderClass"] = df_test_gender["ActualGender"].apply(genderGroups)

    train_texts_gender = getstringtext(df_train_gender)
    test_texts_gender = getstringtext(df_test_gender)

    tfvec = TfidfVectorizer(max_features = 3000)
    tfvec.fit(train_texts_gender)
    tdf_train = tfvec.transform(train_texts_gender).toarray().tolist()
    tdf_test = tfvec.transform(test_texts_gender).toarray().tolist()

    gender_train = df_train_gender["GenderClass"].to_list()
    gender_test = df_test_gender["GenderClass"].to_list()

    logisticRegr_gender = LogisticRegression(penalty=gender_param["k_"+str(i+1)]["params"]["penalty"], C=gender_param["k_"+str(i+1)]["params"]["C"])
    logisticRegr_gender.fit(tdf_train, gender_train)

    pred_gender_test=logisticRegr_gender.predict(tdf_test)

    df_test_gender["PredictGender"] = pred_gender_test

    df_train_fem = df_train_gender[df_train_gender["GenderClass"]==1]
    df_train_masc = df_train_gender[df_train_gender["GenderClass"]==0]

    df_test_fem = df_test_gender[df_test_gender["PredictGender"]==1]
    df_test_masc = df_test_gender[df_test_gender["PredictGender"]==0]

    train_texts_masc = getstringtext(df_train_masc)
    test_texts_masc = getstringtext(df_test_masc)
    tfvec_masc = TfidfVectorizer(max_features = 3000)
    tfvec_masc.fit(train_texts_masc)
    tdf_train_masc = tfvec_masc.transform(train_texts_masc).toarray().tolist()
    tdf_test_masc = tfvec_masc.transform(test_texts_masc).toarray().tolist()
    age_train_masc = df_train_masc["AgeClass"].to_list()
    age_test_masc = df_test_masc["AgeClass"].to_list()
    
    logreg=LogisticRegression()
    logreg_cv=GridSearchCV(logreg,grid,cv=10, scoring='f1_macro')
    logreg_cv.fit(tdf_train_masc, age_train_masc)
    
    param_dict = dict()
    param_dict["params"] =  logreg_cv.best_params_
    param_dict["accuracy"] = logreg_cv.best_score_
    best_param_dict["k_"+str(i+1)] = param_dict

    print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
    print("accuracy :",logreg_cv.best_score_)

end = time.time()
best_param_dict["time"] = formatTime(end-start)

with open(r'C:\Users\alici\Documents\tcc\v3\melhores_param\b5\age_masc.json', 'w') as f:
    json.dump(best_param_dict, f, indent=4)

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1

tuned hpyerparameters :(best parameters)  {'C': 0.001, 'penalty': 'none'}
accuracy : 0.4656835713306301


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1

tuned hpyerparameters :(best parameters)  {'C': 1000.0, 'penalty': 'l2'}
accuracy : 0.5078433669051517


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Pytho

...

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Pytho

tuned hpyerparameters :(best parameters)  {'C': 0.001, 'penalty': 'none'}
accuracy : 0.47707685245610937


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Pytho

tuned hpyerparameters :(best parameters)  {'C': 0.001, 'penalty': 'none'}
accuracy : 0.49453730501213344


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
70 fits failed out of a total of 210.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
70 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\alici\AppData\Local\Program